In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from scipy.optimize import curve_fit
from lmfit import Model, minimize, Parameters, report_fit
from lmfit.models import GaussianModel, PolynomialModel
import astropy.units as u
import pandas as pd
import glob
import pandas as pd
import csv
import celerite

In [4]:
def Gauss(x, h1, c1, w1):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2))
    return G

def doubleGauss(x, h1, c1, w1, h2, c2, w2):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2))
    return G

def tripleGauss(x, h1, c1,w1, h2, c2, w2, h3, c3, w3):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2)) + h3*np.exp(-(x-c3)**2/(2*w3**2))
    return G


In [38]:
def fit_coord(l, b, n_HI = 1, n_65 = 1, n_67 = 1, n_CO = 1, HI_p0 = [1, 0, 1], OH65_p0 = [0.03, 0, 4], OH67_p0 = [0.03, 0, 4], CO_p0 = [1, 0, 1], HI_bounds = (-np.inf, np.inf), OH65_bounds = (-np.inf, np.inf), OH67_bounds = (-np.inf, np.inf), CO_bounds = (-np.inf, np.inf), Cloud = []):
    
    # Setup a pandas dataframe to save all the variables we'd like
    
    '''Purpose:
    This function fits multiple gaussians to HI, CO and HI data and returns the parameters...
    
    Parameters:
    l: galactic latitude, int
    b: galactic lontitude, int
    HIdata: np.array of HI data in Tb, vel
    OHdata: np.array of OH data in Tb, vel
    ...

    Returns a pandas dataframe
    '''
    OH67 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_67basesub.txt')
    OH65 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_65basesub.txt')
    HI = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/HI' + str(l) + 'p' + str(b) + '.txt', data_start = 13, data_end = 167)
    CO = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/CO' + str(l) + 'p' + str(b) + '.txt')



    resultsTable = pd.DataFrame(np.array([[0, 
                                         0, 
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0, 
                                         0,
                                         0,
                                         0]]),
                               columns=['l (deg)', 
                                        'b (deg)', 
                                        'line',
                                        'comp',
                                        'T_peak (K)',
                                        'T_error (K)', 
                                        'CenterVel (km/s)',
                                        'CenterVel_error (km/s)',
                                        'FWHM (km/s)',
                                        'FWHM error (km/s)',
                                        'Cloud'
                                       ])

    velHI = HI['col1']
    TaHI = HI['col2']
    vel67 = OH67['col1']
    Ta67 = OH67['col2']
    vel65 = OH65['col1']
    Ta65 = OH65['col2']
    velCO = CO['col1']
    TaCO = CO['col2']

    #fit HI data
    if n_HI == 1:

        parametersHI, covarianceHI = curve_fit(Gauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0]]
        fitHIc = [parametersHI[1]]
        fitHIw = [parametersHI[2]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0]]
        errorHIc = [perrHI[1]]
        errorHIw = [perrHI[2]]
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        Cloud
                                                        ]

        
    if n_HI == 2:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HI_bounds = np.tile(HI_bounds, n_HI)
        parametersHI, covarianceHI = curve_fit(doubleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0], parametersHI[3]]
        fitHIc = [parametersHI[1], parametersHI[4]]
        fitHIw = [parametersHI[2], parametersHI[5]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3]]
        errorHIc = [perrHI[1], perrHI[4]]
        errorHIw = [perrHI[2], perrHI[5]]
        
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        Cloud
                                                        ]
            
    if n_HI == 3:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HI_bounds = np.tile(HI_bounds, n_HI)
        parametersHI, covarianceHI = curve_fit(tripleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)
        
        fitHIh = [parametersHI[0], parametersHI[3], parametersHI[6]]
        fitHIc = [parametersHI[1], parametersHI[4], parametersHI[7]]
        fitHIw = [parametersHI[2], parametersHI[5], parametersHI[8]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3], perrHI[6]]
        errorHIc = [perrHI[1], perrHI[4], perrHI[7]]
        errorHIw = [perrHI[2], perrHI[5], perrHI[8]]

        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        Cloud
                                                        ]

    #fit 67 data
    
    if n_67 == 1:
        parameters67, covariance67 = curve_fit(Gauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds)
        
        fit67h = [parameters67[0]]
        fit67c = [parameters67[1]]
        fit67w = [parameters67[2]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0]]
        error67c = [perr67[1]]
        error67w = [perr67[2]]
        
        r = n_HI - n_67
        
        
        for i in range(n_67+r):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i+r],
                                                        error67h[i+r],
                                                        fit67c[i+r],
                                                        error67c[i+r],
                                                        fit67w[i+r],
                                                        error67w[i+r],
                                                        Cloud
                                                        ]

        
    if n_67 == 2:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_bounds = np.tile(OH67_bounds, n_67)
        parameters67, covariance67 = curve_fit(doubleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds)

        fit67h = [parameters67[0], parameters67[3]]
        fit67c = [parameters67[1], parameters67[4]]
        fit67w = [parameters67[2], parameters67[5]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3]]
        error67c = [perr67[1], perr67[4]]
        error67w = [perr67[2], perr67[5]]
        
        
        r = n_HI - n_67
            

                
        for i in range(n_67+r):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i+r],
                                                        error67h[i+r],
                                                        fit67c[i+r],
                                                        error67c[i+r],
                                                        fit67w[i+r],
                                                        error67w[i+r],
                                                        Cloud
                                                        ]
            

            
    if n_67 == 3:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_bounds = np.tile(OH67_bounds, n_67)
        parameters67, covariance67 = curve_fit(tripleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds) 
    
        fit67h = [parameters67[0], parameters67[3], parameters67[6]]
        fit67c = [parameters67[1], parameters67[4], parameters67[7]]
        fit67w = [parameters67[2], parameters67[5], parameters67[8]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3], perr67[6]]
        error67c = [perr67[1], perr67[4], perr67[7]]
        error67w = [perr67[2], perr67[5], perr67[8]]

                
        for i in range(n_67+r):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        error67h[i],
                                                        fit67c[i],
                                                        error67c[i],
                                                        fit67w[i],
                                                        error67w[i],
                                                        Cloud
                                                        ]
            
    #fit 65 data
    
    if n_65 == 1:
        parameters65, covariance65 = curve_fit(Gauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)
        
        fit65h = [parameters65[0]]
        fit65c = [parameters65[1]]
        fit65w = [parameters65[2]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0]]
        error65c = [perr65[1]]
        error65w = [perr65[2]]
        
        q = n_OH67 - n65

        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i+q],
                                                        error65h[i+q],
                                                        fit65c[i+q],
                                                        error65c[i+q],
                                                        fit65w[i+q],
                                                        error65w[i+q],
                                                        Cloud
                                                        ]
    
    if n_65 == 2:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_bounds = np.tile(OH65_bounds, n_65)
        parameters65, covariance65 = curve_fit(doubleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)
    
        fit65h = [parameters65[0], parameters65[3]]
        fit65c = [parameters65[1], parameters65[4]]
        fit65w = [parameters65[2], parameters65[5]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0], perr65[3]]
        error65c = [perr65[1], perr65[4]]
        error65w = [perr65[2], perr65[5]]
        
        q = n_OH67 - n65
        
        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i+q],
                                                        error65h[i+q],
                                                        fit65c[i+q],
                                                        error65c[i+q],
                                                        fit65w[i+q],
                                                        error65w[i+q],
                                                        Cloud
                                                        ]

            
    if n_65 == 3:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_bounds = np.tile(OH65_bounds, n_65)
        parameters65, covariance65 = curve_fit(tripleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)    
        
        fit65h = [parameters65[0], parameters65[3], parameters65[6]]
        fit65c = [parameters65[1], parameters65[4], parameters65[7]]
        fit65w = [parameters65[2], parameters65[5], parameters65[8]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0], perr65[3], perr65[6]]
        error65c = [perr65[1], perr65[4], perr65[7]]
        error65w = [perr65[2], perr65[5], perr65[8]]
        
        
        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        error65h[i],
                                                        fit65c[i],
                                                        error65c[i],
                                                        fit65w[i],
                                                        error65w[i],
                                                        Cloud
                                                        ]




            
    #fit CO data
    
    if n_CO == 1:
        parametersCO, covarianceCO = curve_fit(Gauss, velCO, TaCO, CO_p0,bounds =  CO_bounds)
            
        fitCOh = [parametersCO[0]]
        fitCOc = [parametersCO[1]]
        fitCOw = [parametersCO[2]]
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0]]
        errorCOc = [perrCO[1]]
        errorCOw = [perrCO[2]]
               
        s = n_65-n_CO
        
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i+s],
                                                        errorCOh[i+s],
                                                        fitCOc[i+s],
                                                        errorCOc[i+s],
                                                        fitCOw[i+s],
                                                        errorCOw[i+s],
                                                        Cloud
                                                        ]

        
    if n_CO == 2:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        CO_bounds = np.tile(CO_bounds, n_CO)
        parametersCO, covarianceCO = curve_fit(doubleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds)
    
        fitCOh = [parametersCO[0], parametersCO[3]]
        fitCOc = [parametersCO[1], parametersCO[4]]
        fitCOw = [parametersCO[2], parametersCO[5]] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3]]
        errorCOc = [perrCO[1], perrCO[4]]
        errorCOw = [perrCO[2], perrCO[5]]
        
        s = n_65-n_CO
    
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i+s],
                                                        errorCOh[i+s],
                                                        fitCOc[i+s],
                                                        errorCOc[i+s],
                                                        fitCOw[i+s],
                                                        errorCOw[i+s],
                                                        Cloud
                                                        ]

    if n_CO == 3:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        CO_bounds = np.tile(CO_bounds, n_CO)
        parametersCO, covarianceCO = curve_fit(tripleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds) 
            
        fitCOh = [parametersCO[0], parametersCO[3], parametersCO[6]]
        fitCOc = [parametersCO[1], parametersCO[4], parametersCO[7]]
        fitCOw = [parametersCO[2], parametersCO[5], parametersCO[8]] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3], perrCO[6]]
        errorCOc = [perrCO[1], perrCO[4], perrCO[7]]
        errorCOw = [perrCO[2], perrCO[5], perrCO[8]]
        
        
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        errorCOh[i],
                                                        fitCOc[i],
                                                        errorCOc[i],
                                                        fitCOw[i],
                                                        errorCOw[i],
                                                        Cloud
                                                        ]
            

    


               
    resultsTable = resultsTable.drop(0)
    return resultsTable
    #df = fit_coord(...)
    #df.save_csv()

    #files = glob.glob(*.csv)
    #for file in files:
        #df.read_csv(...)
        #df_full.concatenate(df)
        #df_full.summary()
        
    
'''
SETTING UP THE BIG TABLE:

df = fit_coord(l, b, params)
df.to_csv("l+b_fitted.csv")
df

ELSEWHERE (another cell/notebook)

files = glob.glob(fits/*csv)

df_full = pd.df()

#concate each df:
for file in files:
    df.read_csv(file)
    df_full.concatenate(file)
'''

   
        
        
        
        
        
        
        

'\nSETTING UP THE BIG TABLE:\n\ndf = fit_coord(l, b, params)\ndf.to_csv("l+b_fitted.csv")\ndf\n\nELSEWHERE (another cell/notebook)\n\nfiles = glob.glob(fits/*csv)\n\ndf_full = pd.df()\n\n#concate each df:\nfor file in files:\n    df.read_csv(file)\n    df_full.concatenate(file)\n'

In [40]:
fit_coord(119070, 2800, n_HI = 2, n_67 = 1, HI_p0 = (20, 10, 15), HI_bounds = ([0, -20, 1], [30, 30, 30]), Cloud = 'Polaris')

IndexError: list index out of range

In [11]:
#fit_coord(119070, 2800, n_HI = 2)
df = fit_coord(119070, 2800, n_HI = 2, HI_p0 = (20, 10, 15), HI_bounds = ([0, -20, 1], [30, 30, 30]), Cloud = 'Polaris')
df.to_csv("119070p2800_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,119070,2800,HI,1,25.108944,0.262239,-4.629767,0.021005,1.847756,0.024123,Polaris
2,119070,2800,HI,2,6.805908,0.134199,-9.986011,0.218256,11.427903,0.200721,Polaris
3,119070,2800,65,1,0.043520,0.002106,-4.251790,0.060399,1.081164,0.060425,Polaris
4,119070,2800,67,1,0.086602,0.002094,-4.358241,0.030806,1.103239,0.030814,Polaris
5,119070,2800,CO,1,1.613065,0.125558,-3.779779,0.077823,0.865799,0.077823,Polaris


In [12]:
df = fit_coord(119210, 2000, n_HI = 3, n_67 = 2, OH67_p0 = (1, -1, 9), HI_p0 = (1, -8, 30),
          OH67_bounds = ([0, -20, 1], [3, 4, 10]), 
            HI_bounds = ([0, -25, 3], [80, 25, 30]), Cloud = 'Polaris')
df.to_csv("119210p2000_fitted.csv")
df

#work on this fit a bit more

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,119210,2000,HI,1,25.231412,0.552844,-11.512670,0.112793,3.000000,0.102499,Polaris
2,119210,2000,HI,2,21.267959,0.471899,-1.436886,0.157771,4.015097,0.164382,Polaris
3,119210,2000,HI,3,4.384096,0.216440,-24.031410,1.531909,24.926262,1.177009,Polaris
4,119210,2000,65,1,0.015542,0.002313,0.695469,0.242541,1.411649,0.242541,Polaris
5,119210,2000,67,1,0.013802,0.001836,-3.734234,0.707866,2.494765,0.692334,Polaris
6,119210,2000,67,2,0.037613,0.004252,0.560002,0.105356,1.000000,0.123947,Polaris
7,119210,2000,CO,1,0.640718,0.144064,0.650001,0.190147,0.732573,0.190346,Polaris


In [13]:
#fit_coord(120500, 1863, n_HI = 3)
df = fit_coord(120500, 1863, n_HI = 3, HI_p0 = (1, 1, 1), 
          HI_bounds = ([0, -12, 1], [30, 5, 50]), Cloud = 'Polaris')
df.to_csv("120500p1863_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,120500,1863,HI,1,16.151980,1.153345,-9.055389,0.377908,9.278406,0.337238,Polaris
2,120500,1863,HI,2,29.351272,1.058692,-1.140389,0.061317,1.947711,0.081703,Polaris
3,120500,1863,HI,3,7.999107,1.201906,-9.832292,0.209921,1.888819,0.323243,Polaris
4,120500,1863,65,1,0.056665,0.002598,-0.806004,0.046752,0.882485,0.046676,Polaris
5,120500,1863,67,1,0.098978,0.002308,-0.888053,0.026717,0.991228,0.026658,Polaris
6,120500,1863,CO,1,2.971228,0.136213,-0.813725,0.036567,0.690755,0.036565,Polaris


In [14]:
df = fit_coord(120500, 2963, n_67 = 3, OH67_p0 = [1, 0, 2], CO_p0 = [1, -5, 0.5], 
           OH65_p0 = [-1, -8, 1], OH67_bounds = ([0, -10, 0], [3, 5, 5]), 
               OH65_bounds = ([-1, -25, 0], [0, 0, 5]), Cloud = 'Polaris')
df.to_csv("120500p2963_fitted.csv")
df

#look into 65 fit

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,120500,2963,HI,1,22.742624,0.609856,-4.095128,0.146664,4.735093,0.146632,Polaris
2,120500,2963,65,1,-0.083266,0.005933,-8.052769,0.057591,0.176147,0.020393,Polaris
3,120500,2963,67,1,0.030204,0.003289,-6.391249,0.076162,0.627051,0.089858,Polaris
4,120500,2963,67,2,0.014573,0.001365,-3.185736,0.465729,3.041937,0.375118,Polaris
5,120500,2963,67,3,0.002143,0.000000,4.975444,0.000000,0.025524,0.000000,Polaris
6,120500,2963,CO,1,1.475331,0.128105,-6.307529,0.057041,0.563438,0.056081,Polaris


In [15]:
#fit_coord(121070, 2175, n_65 = 3, n_67 = 2, n_CO = 2, n_HI = 2, 
          #CO_bounds = ([0, -10, 0, 0, -10, 0],[5, 1, 5, 5, 1, 5]))
df = fit_coord(121070, 2175, n_65 = 3, n_67 = 2, n_CO = 2, n_HI = 2,
           OH65_p0 = (1, -2, 4),
           CO_bounds = ([0, -10, 0], [5, 1, 5]),
          HI_bounds = ([0, -15, 0], [20, 5, 30]), 
           OH67_bounds = ([0, -10, 0], [10, 5, 10]), 
           OH65_bounds = ([0, -10, 0], [10, 5, 10]), Cloud = 'Polaris')
df.to_csv("121070p2175_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,121070,2175,HI,1,9.199653,0.514994,-2.795865,0.134951,2.573746,0.172622,Polaris
2,121070,2175,HI,2,9.894485,0.414983,-6.992074,0.301406,9.146891,0.253433,Polaris
3,121070,2175,65,1,0.077627,0.003124,-5.853875,0.040864,0.624954,0.037381,Polaris
4,121070,2175,65,2,0.016278,0.002113,-2.681926,0.275916,1.424557,0.306215,Polaris
5,121070,2175,65,3,0.143523,0.000000,-0.422853,0.000000,0.000543,0.000000,Polaris
6,121070,2175,67,1,0.122156,0.004859,-6.000257,0.022377,0.666130,0.044360,Polaris
7,121070,2175,67,2,0.036667,0.001859,-3.012162,0.276465,2.051294,0.236952,Polaris
8,121070,2175,CO,1,3.861257,0.274409,-6.122722,0.043057,0.780493,0.048677,Polaris
9,121070,2175,CO,2,1.935918,0.091977,-3.416228,0.234138,1.650391,0.196384,Polaris


In [16]:
df = fit_coord(121930, 1975, n_HI = 2, n_67 = 2, OH67_p0 = (1, 0, 8), HI_p0 = (1.3, -10, 4),
           OH65_p0 = (1, 0, 2), 
           OH65_bounds = ([0, -5, .1], [1, 10, 5]), 
           OH67_bounds = ([0, -10, .1], [20, 5, 20]), 
           HI_bounds = ([0, -20, 1], [40, 5, 50]), Cloud = 'Polaris') 

df.to_csv("121930p1975_fitted.csv")
df
#check later
#high 65 error

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,121930,1975,HI,1,19.510653,1.101174,-0.612584,0.303870,3.564322,0.254745,Polaris
2,121930,1975,HI,2,18.323957,0.657323,-12.521182,0.460614,5.477668,0.425020,Polaris
3,121930,1975,65,1,0.065813,2254.990334,1.298229,154.264650,0.245517,2361.243267,Polaris
4,121930,1975,67,1,0.015215,0.002820,-2.997574,0.160875,0.717521,0.149671,Polaris
5,121930,1975,67,2,0.118112,1300.779045,1.323984,10.731620,0.249545,798.576881,Polaris
6,121930,1975,CO,1,0.859033,0.160622,1.332860,0.119604,0.569361,0.125836,Polaris


In [17]:
df = fit_coord(122220, 1875, n_HI = 3, HI_p0 = (1, -10, 25), CO_p0 = (1, 2, 0.5),
          CO_bounds = ([0, 0, 0.1], [3, 3, 3]), 
          HI_bounds = ([0, -20, 1], [30, 20, 50]), Cloud = 'Polaris')
df.to_csv("122220p1875_fitted.csv")
df
#CHECK LATER ON CO ERROR (basically just noise as seen in plot)

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,122220,1875,HI,1,10.443772,1.629264,-12.628355,0.225800,1.316190,0.258288,Polaris
2,122220,1875,HI,2,19.828186,1.001218,-8.541770,0.541676,9.482401,0.378219,Polaris
3,122220,1875,HI,3,5.199265,1.425286,0.066284,0.683497,2.496087,0.858417,Polaris
4,122220,1875,65,1,0.013580,0.001910,1.981341,0.187442,1.154046,0.187441,Polaris
5,122220,1875,67,1,0.027175,0.002403,1.999606,0.082510,0.803392,0.081722,Polaris
6,122220,1875,CO,1,2.180064,4531.473308,1.964676,19.727884,0.192040,139.530787,Polaris


In [18]:
df = fit_coord(122500, 2900, n_HI = 2, n_67 = 3, 
           HI_p0 = (1, -10, 25), OH67_p0 = (1, -3, 4),
           CO_p0 = (2, -4, 3),
           OH67_bounds = ([0, -8, 1], [5, 3, 10]), 
           CO_bounds = ([0, -10, 1], [5, -1, 5]), 
          HI_bounds = ([0, -30, 1], [20, -3, 30]), Cloud = 'Polaris')
df.to_csv = ("122500p2900_fitted.csv")
df
#issue with OH bounds and CO 

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,122500,2900,HI,1,17.644284,0.443362,-5.238369,0.119844,3.936437,0.125042,Polaris
2,122500,2900,HI,2,9.587305,0.401926,-22.515768,0.243528,4.879128,0.259805,Polaris
3,122500,2900,65,1,0.012392,0.001657,-6.669039,0.205526,1.331156,0.205526,Polaris
4,122500,2900,67,1,0.011433,0.001886,-3.772149,0.226735,1.000000,0.257764,Polaris
5,122500,2900,67,2,0.006047,0.001856,0.409318,0.361263,1.000000,0.378499,Polaris
6,122500,2900,67,3,0.015803,0.001864,-7.186640,0.164990,1.000000,0.175135,Polaris
7,122500,2900,CO,1,0.314455,0.075126,-5.528401,0.451912,1.638152,0.451912,Polaris


In [19]:
df = fit_coord(123360, 2075, n_HI = 3, HI_p0 = (2, -3, 1), n_65 = 2, OH65_p0 = (1, 0, 2), 
           n_67 = 3, OH67_p0 = (1, 0, 4), n_CO = 2, CO_p0 = (1, 0, 3)
          , OH67_bounds = ([0, -10, 0], [4, 5, 10]), OH65_bounds = ([0, -6, 0], [1, 3.8, 4]),
          CO_bounds = ([0, -5, 0], [4, 4, 5]), HI_bounds = ([0, -20, 0], [30, 20, 10]), Cloud = 'Polaris')
df.to_csv = ("123360p2075_fitted.csv")
df

#67 temperature peak questionable

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,123360,2075,HI,1,25.697443,1.364047,-3.592407,0.114687,1.981046,0.135200,Polaris
2,123360,2075,HI,2,19.038661,0.766518,4.564557,0.175270,3.071973,0.192284,Polaris
3,123360,2075,HI,3,18.089185,0.598028,-12.401738,0.341516,5.083314,0.339971,Polaris
4,123360,2075,65,1,0.020388,0.001622,1.480527,0.255096,2.777031,0.255096,Polaris
5,123360,2075,65,2,0.009338,0.000000,1.399268,0.000000,0.007393,0.000000,Polaris
6,123360,2075,67,1,0.026694,0.001526,-0.503281,0.190131,1.787836,0.198821,Polaris
7,123360,2075,67,2,0.006208,0.002309,-6.803912,0.311188,0.690371,0.295108,Polaris
8,123360,2075,67,3,0.064067,0.002954,2.865527,0.039473,0.738851,0.042647,Polaris
9,123360,2075,CO,1,1.623683,0.091746,-0.481580,0.086190,1.122112,0.094523,Polaris
10,123360,2075,CO,2,1.423821,0.106481,2.725451,0.083826,0.792736,0.084507,Polaris


In [20]:
df = fit_coord(123500, 1963, n_67 = 2, n_HI = 3, n_CO = 2, HI_p0 = (15, 0, 5), OH65_p0 = (2, 0, 2),
           CO_p0 = (.4, -2, .4), CO_bounds = ([0, -10, 0], [2, 0, 10]), OH67_p0 = (1, 0, 2), 
          OH65_bounds = ([0, -10, 0], [5, 1, 5]), 
           HI_bounds= ([10, -30, 1], [80, 25, 15]), OH67_bounds = ([0, -10, 0], [5, 1, 5]), Cloud = 'Polaris')

df.to_csv("123500p1963_fitted.csv")
df
#cant get 65 to fit

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,123500,1963,HI,1,18.089110,0.598024,-12.401713,0.341525,5.083383,0.339978,Polaris
2,123500,1963,HI,2,25.697267,1.364069,-3.592402,0.114687,1.981038,0.135201,Polaris
3,123500,1963,HI,3,19.038646,0.766519,4.564559,0.175269,3.071973,0.192284,Polaris
4,123500,1963,65,1,0.017177,122302.577921,0.303674,44284.079547,0.220129,353851.387838,Polaris
5,123500,1963,67,1,0.004447,0.001032,1.000000,1.983459,5.000000,1.395925,Polaris
6,123500,1963,67,2,0.032438,0.002206,-4.054945,0.083957,1.327152,0.110279,Polaris
7,123500,1963,CO,1,0.405409,0.219062,-5.356204,0.223274,0.466562,0.298617,Polaris
8,123500,1963,CO,2,0.527746,0.186118,-4.651607,0.416166,1.433618,0.298320,Polaris


In [21]:
df = fit_coord(125070, 1950, n_HI = 2, HI_p0 = (10, 1, 20), 
               HI_bounds = ([0, -30, 0], [20, 10, 30]), Cloud = 'Polaris')
df.to_csv("125070p1950_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,125070,1950,HI,1,8.403221,0.719006,4.079471,0.195456,3.464665,0.308162,Polaris
2,125070,1950,HI,2,16.459169,0.269667,-8.096963,0.399496,10.082047,0.284664,Polaris
3,125070,1950,65,1,0.059559,0.002486,0.686141,0.038342,0.555600,0.025695,Polaris
4,125070,1950,67,1,0.106404,0.002872,0.660790,0.022660,0.620396,0.018588,Polaris
5,125070,1950,CO,1,1.962675,0.147545,0.680557,0.051534,0.601743,0.052906,Polaris


In [22]:
df = fit_coord(125220, 3250, OH67_bounds = ([0, -5, 0], [3, 5, 5]), Cloud = 'Polaris')
df.to_csv("125220p3250_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,125220,3250,HI,1,36.662619,0.776355,1.152745,0.067798,2.775750,0.067794,Polaris
2,125220,3250,65,1,0.068332,0.002120,0.234129,0.034403,0.961648,0.034518,Polaris
3,125220,3250,67,1,0.120393,0.002196,0.069041,0.022092,1.048728,0.022080,Polaris
4,125220,3250,CO,1,0.863866,0.103696,0.389683,0.137698,0.993339,0.137698,Polaris


In [24]:
df = fit_coord(125500, 2200, n_HI = 2, n_67 = 2, n_65 = 2, OH67_p0 = (1, 0, 2)
          , OH67_bounds = ([0, -5, 0], [1.5, 5, 5]), HI_bounds = ([0, -20, 0], [20, 15, 30]),
          OH65_bounds = ([0, -5, 0], [1.5, 5, 5]), Cloud = 'Polaris') 
df.to_csv("125500p2200_fitted.csv")
df

#note, double check this is in polaris later!!

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,125500,2200,HI,1,14.435097,0.377697,-2.672325,0.092363,3.392045,0.111443,Polaris
2,125500,2200,HI,2,6.079058,0.242600,-9.139501,0.554362,16.841603,0.536627,Polaris
3,125500,2200,65,1,0.009467,0.002534,-3.343954,0.598885,2.205917,0.426936,Polaris
4,125500,2200,65,2,0.029502,0.003611,-4.582642,0.082595,0.594416,0.099486,Polaris
5,125500,2200,67,1,0.004448,0.004521,2.728866,0.000244,0.086668,0.000256,Polaris
6,125500,2200,67,2,0.063852,0.002242,-4.560828,0.043307,1.067873,0.043303,Polaris
7,125500,2200,CO,1,2.437001,0.095121,-4.804518,0.038853,0.911051,0.043082,Polaris


In [23]:
df = fit_coord(127500, 2088, n_67 = 2, n_65 = 2, 
           n_HI = 3, OH67_p0 = (4, 0, 3), 
           OH65_p0 = (2, 0, 3), 
           HI_p0 = (10, 0, 30), 
           OH65_bounds = ([0, -15, 1], [2, 10, 10]), 
           OH67_bounds = ([0, -15, 1], [15, 5, 10]),
          HI_bounds = ([0, -25, 1], [30, 18, 40]), Cloud = 'Polaris')
df.to_csv("127500p2088_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,127500,2088,HI,1,24.285441,0.601601,-2.632001,0.066571,2.669135,0.072193,Polaris
2,127500,2088,HI,2,16.973269,0.247301,6.196843,0.135649,4.078036,0.123679,Polaris
3,127500,2088,HI,3,11.197022,0.173359,-13.794739,0.298294,7.508597,0.269080,Polaris
4,127500,2088,65,1,0.005158,0.001388,9.148213,0.911445,2.900223,0.955188,Polaris
5,127500,2088,65,2,0.025791,0.001588,-1.390457,0.159228,2.183248,0.163984,Polaris
6,127500,2088,67,1,0.006635,0.001454,3.496465,1.669065,6.693866,1.167051,Polaris
7,127500,2088,67,2,0.049094,0.002290,-1.098453,0.082017,1.737858,0.100797,Polaris
8,127500,2088,CO,1,2.598015,0.103059,-0.610891,0.059003,1.288129,0.059003,Polaris


In [74]:
df = fit_coord(137160, 4300, OH67_p0 = (1, 10, 1), n_HI = 2,
          HI_bounds = ([0, 0, 1], [20 ,15, 15]), Cloud = 'Ursa Major')
df.to_csv("137160p4300_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,137160,4300,HI,1,2.592075e+00,1.587096e-01,2.875547e-16,8.080451e-01,10.876046,6.581775e-01
2,137160,4300,HI,2,1.121861e+01,3.570247e-01,7.849033e+00,6.140464e-02,1.756083,6.965312e-02
3,137160,4300,65,1,3.101224e-03,1.551295e-03,-1.008849e+00,9.413538e-01,1.629755,9.413538e-01
4,137160,4300,67,1,1.004865e-02,2.068852e-03,8.702887e+00,1.959348e-01,0.823589,1.956942e-01
5,137160,4300,CO,1,-1.617513e-19,2.212943e-20,1.130883e-09,1.319636e-10,1.000000,8.437336e-10


In [75]:
df = fit_coord(142210, 2325, n_HI = 3, HI_p0 = (1, 0, 8), 
           HI_bounds = ([0, -30, 0], [20, 5, 50]), Cloud = 'Camelopardalis')
df.to_csv("142210p2325_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,142210,2325,HI,1,0.818782,0.095787,-28.850766,5.007798,36.470092,3.877945
2,142210,2325,HI,2,14.290681,0.283359,-3.126089,0.110026,8.025420,0.147917
3,142210,2325,HI,3,19.395988,0.384563,-1.330761,0.033308,1.744758,0.042756
4,142210,2325,65,1,0.043649,0.002481,-1.955868,0.053688,0.789529,0.050298
5,142210,2325,67,1,0.084863,0.002742,-2.042813,0.031121,0.820334,0.030163
6,142210,2325,CO,1,2.504879,0.131268,-2.169310,0.043053,0.711052,0.043011


In [76]:
df = fit_coord(143000, 3850, n_HI = 2, HI_bounds = ([0, -10, 0], [20, 15, 30]), Cloud = 'Ursa Major')
df.to_csv("143000p3850_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,143000,3850,HI,1,12.805758,1.821154,2.037586,0.150874,1.359361,0.202025
2,143000,3850,HI,2,17.597695,1.611608,4.112797,0.276431,3.666836,0.175858
3,143000,3850,65,1,0.053961,0.002037,2.292664,0.039967,0.921137,0.040368
4,143000,3850,67,1,0.097741,0.002296,2.225671,0.024816,0.918526,0.025030
5,143000,3850,CO,1,2.491926,0.101556,2.302071,0.037419,0.795076,0.037414


In [77]:
df = fit_coord(146070, 1775, n_HI = 2, OH67_bounds = ([0, -5, 0], [20, 5, 5])
          , OH65_bounds = ([0, -5, 0], [20, 5, 5]), HI_bounds = ([0, -20, 0], [20, 8, 30]), Cloud = 'Camelopardalis')
df.to_csv("146070p1775_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,146070,1775,HI,1,17.389566,0.882948,-1.931734,0.210942,5.902981,0.168969
2,146070,1775,HI,2,20.000000,0.976255,0.743239,0.078732,1.976837,0.105158
3,146070,1775,65,1,0.169170,0.002339,0.591286,0.012319,0.687076,0.010227
4,146070,1775,67,1,0.230267,0.002680,0.487612,0.010358,0.723211,0.009325
5,146070,1775,CO,1,4.350812,0.114572,0.258126,0.027344,0.899220,0.027344


In [78]:
df = fit_coord(146200, 3963, n_HI = 2, HI_p0 = (1, 0, 5), HI_bounds = ([0, -10, 0], [20, 15, 30]), Cloud = 'Ursa Major')
df.to_csv("146200p3963_fitted.csv")
df

#weird HI bounds behavior when varying upper bounds

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,146200,3963,HI,1,9.588168,0.809131,-2.501404,0.378141,3.087574,0.413438
2,146200,3963,HI,2,15.570900,1.043702,5.265058,0.178000,1.844490,0.173670
3,146200,3963,65,1,0.017295,0.003077,0.399105,0.124299,0.695627,0.154214
4,146200,3963,67,1,0.022717,0.001933,-0.098259,0.122012,1.241909,0.122010
5,146200,3963,CO,1,1.779890,0.092886,0.001515,0.049196,0.816434,0.049200


In [79]:
df = fit_coord(146930, 2075, Cloud = 'Camelopardalis')
df.to_csv("146930p2075_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,146930,2075,HI,1,37.110928,0.772866,-1.210916,0.072475,3.014247,0.072489
2,146930,2075,65,1,0.033501,0.009247,0.136408,0.152707,0.642267,0.228113
3,146930,2075,67,1,0.057882,0.005701,-0.134523,0.096376,0.841744,0.095200
4,146930,2075,CO,1,0.697665,0.092378,0.055885,0.106260,0.695648,0.106522


In [80]:
df = fit_coord(147000, 4038, n_CO = 2, n_67 = 2, n_65 = 2, OH67_bounds = ([0, -15, 0], [1, 3, 5])
          , OH65_bounds = ([0, -5, 0], [1, 3, 5]), CO_bounds = ([0, -8, 0], [2, 5, 5]), Cloud = 'Ursa Major')
df.to_csv("147000p4038_fitted.csv")
df


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,147000,4038,HI,1,11.355956,0.258733,0.643146,0.131294,4.992655,0.131359
2,147000,4038,65,1,0.008834,0.002399,-0.918987,0.567626,3.257373,0.706333
3,147000,4038,65,2,0.016684,0.003595,-1.267395,0.147424,0.593566,0.184706
4,147000,4038,67,1,0.007669,0.002651,1.023450,0.545261,0.931344,0.540849
5,147000,4038,67,2,0.041632,0.002633,-1.883506,0.100896,0.942067,0.100066
6,147000,4038,CO,1,1.542826,0.100167,-1.863680,0.063116,0.841038,0.064971
7,147000,4038,CO,2,0.607147,0.128383,1.116347,0.126575,0.495881,0.123451


In [82]:
df = fit_coord(147200, 4075, n_67 = 2, n_65 = 1, n_CO = 2, 
           OH65_bounds = ([0, -15, 0], [2, 8, 5]),
           OH67_bounds = ([0, -15, 0], [2, 8, 5]),
          CO_bounds = ([0, 0, 0], [2, 10, 2]), Cloud = 'Ursa Major')  
df.to_csv("147200p4075_fitted.csv")
df

#issue when I apply bounds to 65

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,147200,4075,HI,1,19.663554,0.344352,1.742909,0.076426,3.782216,0.076468
2,147200,4075,65,1,0.054946,0.002747,3.812877,0.040619,0.683667,0.039374
3,147200,4075,67,1,0.076728,0.002021,3.683673,0.023821,0.731117,0.021176
4,147200,4075,67,2,0.025082,0.001708,-1.748481,0.088044,1.119474,0.088059
5,147200,4075,CO,1,1.999999,54.077501,3.789920,1.946731,0.689066,1.123626
6,147200,4075,CO,2,1.999999,54.099328,3.668363,1.484570,0.603640,1.225258


In [83]:
df = fit_coord(148200, 3863, n_HI = 2, HI_p0 = (1, 0, 5), 
           HI_bounds = ([0, -10, 1], [20, 10, 20]), Cloud = 'Ursa Major')
df.to_csv("148200p3863_fitted.csv")
df


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,148200,3863,HI,1,8.934638,1.033549,1.100517,0.460988,5.912545,0.459897
2,148200,3863,HI,2,7.812903,1.254322,-0.933010,0.231440,1.572206,0.301958
3,148200,3863,65,1,0.041634,0.002530,-2.200168,0.060638,0.860289,0.060124
4,148200,3863,67,1,0.047720,0.002155,-1.827805,0.062944,1.207130,0.062943
5,148200,3863,CO,1,1.576454,0.102998,-2.316782,0.052379,0.693315,0.052232


In [85]:
df = fit_coord(148210, 2350, n_HI = 2, HI_p0 = (1, 0, 5), HI_bounds = ([0, -20, 0], [20, 15, 30]),
           OH67_p0 = (1, 1, 1),
          OH67_bounds = ([0, 0, 0], [1, 5, 2]), Cloud = 'Camelopardalis')
df.to_csv("148210p2350_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,148210,2350,HI,1,11.950775,0.233022,-1.813379,0.157706,8.475079,0.148828
2,148210,2350,HI,2,10.816934,0.469056,1.905390,0.061185,1.293286,0.070372
3,148210,2350,65,1,0.044365,0.003695,1.283639,0.064017,0.609142,0.059267
4,148210,2350,67,1,0.066329,0.003010,1.370475,0.038548,0.671339,0.033823
5,148210,2350,CO,1,3.165647,0.141303,1.304324,0.026811,0.545670,0.029165


In [86]:
df = fit_coord(150120, 4067, Cloud = 'Ursa Major')
df.to_csv("150120p4067_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,150120,4067,HI,1,1.060897e+01,2.619626e-01,1.221581e+00,1.480027e-01,5.192746,1.480601e-01
2,150120,4067,65,1,2.465818e-03,9.400724e-04,1.338215e+00,2.449608e+00,5.567353,2.449608e+00
3,150120,4067,67,1,3.396686e-03,1.080381e-03,-3.658921e+00,1.169826e+00,3.183563,1.169826e+00
4,150120,4067,CO,1,-1.617513e-19,2.212943e-20,1.130883e-09,1.319636e-10,1.000000,8.437336e-10


In [87]:
df = fit_coord(151500, 1925, n_HI = 2, HI_bounds = ([0, -15, 0], [20, 15, 30]), Cloud = 'Camelopardalis')
df.to_csv("151500p1925_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,151500,1925,HI,1,20.000000,1.010966,-0.746892,0.089164,1.846432,0.115768
2,151500,1925,HI,2,13.944777,0.757857,-1.122311,0.279187,8.748602,0.360982
3,151500,1925,65,1,0.011849,0.002383,-1.449924,0.192803,0.812572,0.185016
4,151500,1925,67,1,0.022660,0.001887,-1.235308,0.124834,1.298628,0.124834
5,151500,1925,CO,1,1.513998,0.124205,-1.399332,0.066468,0.702123,0.066562


In [88]:
df = fit_coord(156540, 3513, n_67 = 2, n_65 = 2, OH67_p0 = (1, 0, 3), OH65_p0 = (1, 0, 3)
          ,OH67_bounds = ([0, -10, 0], [1, 2, 5]), OH65_bounds = ([0, -10, 0], [5, 2, 5]),
          CO_bounds = ([0, -10, 0], [2, 0, 5]), Cloud = 'Ursa Major')
df.to_csv("156540p3513_fitted.csv")
df
#weird CO behavior when bounds applied

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,156540,3513,HI,1,15.205414,0.417686,-0.789128,0.116018,3.658962,0.116042
2,156540,3513,65,1,0.022312,0.002215,-3.047333,0.110732,0.781592,0.099891
3,156540,3513,65,2,0.006454,0.001672,0.436323,0.513166,1.418429,0.567612
4,156540,3513,67,1,0.017449,0.002465,0.426706,0.103879,0.666742,0.115561
5,156540,3513,67,2,0.032019,0.002018,-3.067127,0.064991,0.883918,0.065264
6,156540,3513,CO,1,1.725987,0.114585,-3.363716,0.052656,0.687417,0.052742


In [89]:
df = fit_coord(156640, 3250, Cloud = 'Ursa Major')
df.to_csv("156640p3250_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,156640,3250,HI,1,23.456820,0.679447,-0.742946,0.095651,2.861469,0.095685
2,156640,3250,65,1,0.002425,0.000994,0.949288,1.781178,3.763955,1.781178
3,156640,3250,67,1,0.001190,0.001536,-0.204544,3.025401,-2.028822,3.025401
4,156640,3250,CO,1,1.742431,0.101835,-0.815809,0.068763,1.018795,0.068763


In [20]:
#fit_coord(157350, 2200, n_HI = 3, HI_p0 = (1, 0, 20)) null

In [29]:
files = glob.glob("C:/Users/seven/OneDrive/Documents/CO Dark Final csv/*csv*")

In [30]:
for file in files:
    df.read_csv(file)
    df_full.concatenate(file)
    df_full.summary
